In [27]:
import pandas as pd
import folium
import branca.colormap as cm

# --------------------------------------------------------------------
# 1) Read your CSV
# --------------------------------------------------------------------
df = pd.read_csv("miete_trier50km_detailed2.csv")

# Drop rows without necessary columns
df = df.dropna(subset=["Latitude", "Longitude", "Preis_pro_qm", "Link"])

# --------------------------------------------------------------------
# 2) Remove or clip extreme outliers in 'Preis_pro_qm'
#    Example: remove properties above 95th percentile
# --------------------------------------------------------------------
upper_bound = df["Preis_pro_qm"].quantile(0.95)
df = df[df["Preis_pro_qm"] <= upper_bound]

# --------------------------------------------------------------------
# 3) Compute new min/max after clipping
# --------------------------------------------------------------------
min_price_qm = df["Preis_pro_qm"].min()
max_price_qm = df["Preis_pro_qm"].max()

# --------------------------------------------------------------------
# 4) Create a color scale
# --------------------------------------------------------------------
colormap = cm.LinearColormap(
    colors=["green", "yellow", "red"],
    vmin=min_price_qm,
    vmax=max_price_qm,
    caption="Price per m² (Clipped)"
)

# --------------------------------------------------------------------
# 5) Create Folium map
# --------------------------------------------------------------------
m = folium.Map(location=[49.75, 6.65], zoom_start=12)

# --------------------------------------------------------------------
# 6) Add Circle Markers with popup info
# --------------------------------------------------------------------
for _, row in df.iterrows():
    lat = row["Latitude"]
    lon = row["Longitude"]
    price_qm = row["Preis_pro_qm"]
    link = row["Link"]
    
    # Map Preis_pro_qm to a color
    color = colormap(price_qm)

    # HTML popup with link
    popup_html = (
        f"<b>Price/m²:</b> {price_qm:.2f} €<br>"
        f"<b>Total Price:</b> {row.get('Preis_cleaned', 'n/a')} €<br>"
        f"<b>Zimmer:</b> {row.get('Zimmer', 'n/a')}<br>"
        f"<b>Link:</b> <a href='{link}' target='_blank'>Open Listing</a>"
    )

    folium.CircleMarker(
        location=[lat, lon],
        radius=5,
        weight=1,
        fill=True,
        fill_opacity=0.8,
        color=color,
        fill_color=color,
        popup=popup_html
    ).add_to(m)

# --------------------------------------------------------------------
# 7) Add color legend and save
# --------------------------------------------------------------------
colormap.add_to(m)
m.save("map_markers_price_clipped_with_links.html")
print("Map with clickable links saved as 'map_markers_price_clipped_with_links.html'")


Map with clickable links saved as 'map_markers_price_clipped_with_links.html'
